# Table of Contents

1. ARIMA Model
2. Facebook Prophet Model
3. LSTM

# 1. ARIMA Model
The ARIMA(p,I,q) model has 3 hyperparameters:
- The order of the AR process, based on autocorrelations between past and
present values, denoted by p
- The order of the MA process, based on correlations between past errors and
present values denoted by q
- The order of integration denoted by I (or d in some notations)
- Additional “I” building block in ARIMA, which stands for automatic differencing of non-stationary time series.
> If a time series is not stationary, you can make it stationary by
applying differencing: replacing the actual values by the difference between the actual
and the previous value.

**Step 1.** Get Data
- Loading the transactions data from sales dataset
- Converting it in datetime format

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
from statsmodels.tsa.arima_model import ARIMA

# Data Visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
pio.templates.default = "none"


# Load data
df_trans = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')

# Convert to datetime
df_trans['date'] = pd.to_datetime(df_trans['date'], format = "%Y-%m-%d")
df_trans.head()

**Step 2.** Visualize Data

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(20,8))
df_trans.plot.line(x="date", y="transactions",
                 ax = axes,
#                  rot=0
                )
plt.show()

The chart looks to overwhelming to understand since there are too many data points. It will be easier to group these data, say by week or month. The aggregation will be made by **mean.**

In [ ]:
# Create function to group by the called frequency (W = week, M = month, Y = year)
def grouped(df, key, freq, col):
    """ GROUP DATA WITH CERTAIN FREQUENCY """
    df_grouped = df.groupby([pd.Grouper(key=key, freq=freq)]).agg(mean = (col, 'mean'))
    df_grouped = df_grouped.reset_index()
    return df_grouped

df_grouped_trans_m = grouped(df_trans, 'date', 'M', 'transactions')
df_grouped_trans_w = grouped(df_trans, 'date', 'W', 'transactions')
df_grouped_trans_w.head()

In [ ]:
fig = plt.figure(figsize = (20, 8))
sns.lineplot(x = df_grouped_trans_w['date'], y= df_grouped_trans_w['mean'], linewidth=2, )
plt.title("Average Daily Sales per Week",
          fontsize = 18,
          fontweight = 'bold',
          fontfamily = 'serif',
          loc = 'center')

In [ ]:
fig = plt.figure(figsize = (20, 8))
sns.lineplot(x = df_grouped_trans_m['date'], y= df_grouped_trans_m['mean'], linewidth=2, )
plt.title("Average Daily Sales per Month",
          fontsize = 18,
          fontweight = 'bold',
          fontfamily = 'serif',
          loc = 'center')

**Step 3.** Split to Train and Test data

In [ ]:
print(df_grouped_trans_m.shape)

# first 36 rows as training
train = df_grouped_trans_m[0:36]

# remaining rows for testing
test = df_grouped_trans_m[36:]

print(train.shape, test.shape)

In [ ]:
train.head()

In [ ]:
test

**Step 4.** ARIMA model

- `auto_arima()` function will be used to automatically select the best parameters for an ARIMA model. It takes several parameters to set a range of values for p, d, q, P, D, Q that the function will explore. For example, `start_p=1`, `start_q=1` and `max_p=3`, `max_q=3` are set as the range for p and q.
- The `auto_arima()` model will use the `stepwise=True` option to fit the model iteratively and improve the model at each step.
- The fitted model is then stored in the `model_fit` variable and the summary of the model is printed.
- Finally, the code uses the `predict()` function of the fitted model to forecast the next 'n' periods of the time series.

In [ ]:
pip install pmdarima --quiet

In [ ]:
from pmdarima import auto_arima
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Run combinations of ARIMA(p,d,q)
model_fit = auto_arima(train['mean'],
                       m=12,
                       d=0,
                       D=0,
                       max_order=None,                       
                       max_p=7,
                       max_q=7,
                       max_d=2,
                       max_P=4,
                       max_Q=4,
                       max_D=2,
                       maxiter = 50,
                       alpha = 0.05,
                       n_jobs = -1,
                       seasonal=True,
                       trace=True,
                       error_action='ignore',  
                       suppress_warnings=True, 
                       stepwise=True
                      )

model_fit.summary()

The best ARIMA model that has the lowest AIC is as follows:

In [ ]:
model_fit

**Step 5.** Forecasts on training data given the optimal parameters from **auto_arima**

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Fit the ARIMA model
model_ARIMA = ARIMA(train['mean'],
              order=(1,0,0),
              seasonal_order=(2, 0, 0, 12)
            )

# Fit the model
model_ARIMA = model_ARIMA.fit()

train_forecast = train.copy()
test_forecast = test.copy()

train_forecast['forecast_ARIMA'] = model_ARIMA.predict()
train_forecast[['mean','forecast_ARIMA']].plot(figsize=(20,8))

**Step 6.** Forecast on testing data

In [ ]:
# Forecast and compare against test data

# start date
start = len(train)

# End date
end = len(train)+len(test)-1

test_forecast['forecast_ARIMA'] = model_ARIMA.predict(start=start, end=end, )
test_forecast[['mean','forecast_ARIMA']].plot(figsize=(20,8))

To evaluate the performance of the ARIMA model, the **RMSE** and **MAE** can be computed and these shall serve as the benchmark for other models.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

print("RMSE of Auto ARIMA:", np.sqrt(mean_squared_error(test_forecast['mean'], test_forecast['forecast_ARIMA'])))
print("MAE of Auto ARIMA:", mean_absolute_error(test_forecast['mean'], test_forecast['forecast_ARIMA']))

# 2. Facebook Prophet

- Both are powerful tools for time series forecasting, but have different strengths and weaknesses.
- Prophet is a more user-friendly and flexible model than ARIMA, and it can handle missing data, seasonality, and changepoints automatically. It also provides clear visualizations of the model components.

- On the other hand, ARIMA requires more manual tuning of the model parameters and is more sensitive to outliers. It also requires a deeper understanding of the underlying statistical concepts and can be more difficult to interpret. However, ARIMA can be more flexible as it can be extended to SARIMAX model that allows the addition of exogenous variables.

- It's important to note that the performance of the model depends on the nature of the data and the task at hand. Both Prophet and ARIMA can be used to achieve good performance on time series forecasting problems, but it's important to try multiple models and techniques, evaluate their performance, and select the best one for your specific dataset.

In [ ]:
from prophet import Prophet

Rename column names to 'ds' and 'y'. This is required.

In [ ]:
train_fb = train.copy()
test_fb = test.copy()

train_fb.columns = ['ds','y']
test_fb.columns = ['ds','y']

train_fb.head()

There are two other great plots available when using the Prophet model. The first
one is showing the forecast against the observed data points for the past and future data.

In [ ]:
model_prophet = Prophet()
model_prophet.fit(train_fb)

forecast = model_prophet.predict()
fig = model_prophet.plot(forecast)

The second plot that you can obtain from the Prophet model is a decomposition of
the different impacts of the model. This means that the decomposition can show you the
impact of the different seasonalities at each time step.

In [ ]:
forecast = model_prophet.predict(train_fb)
model_prophet.plot_components(forecast)

In [ ]:
train_forecast['forecast_prophet'] = forecast['yhat']
train_forecast[['mean', 'forecast_ARIMA','forecast_prophet']].plot(figsize=(20,8))

In [ ]:
future_data = model_prophet.make_future_dataframe(periods=20, freq='M')
forecast = model_prophet.predict(future_data)

test_forecast['forecast_prophet'] = forecast[36:]['yhat']
test_forecast[['mean','forecast_ARIMA', 'forecast_prophet']].plot(figsize=(20,8))

To compare the errors between ARIMA vs Facebook Prophet model

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

print("RMSE of Auto ARIMA:", np.sqrt(mean_squared_error(test_forecast['mean'], test_forecast['forecast_ARIMA'])))
print("MAE of Auto ARIMA:", mean_absolute_error(test_forecast['mean'], test_forecast['forecast_ARIMA']))

print("---------------------------------------------\nRMSE of Facebook Prophet:", np.sqrt(mean_squared_error(test_forecast['mean'], test_forecast['forecast_prophet'])))
print("MAE of Facebook Prophet:", mean_absolute_error(test_forecast['mean'], test_forecast['forecast_prophet']))

**Insights:**

Although Facebook Prophet performed better in the training data, it was revealed to perform poorly when validated on testing data. Upon observation, the declining trend made the deviation even worse in the following years. Overfitting may be a concern here.  

# 3. Long Short-Term Memory (LSTMs)

Long Short-Term Memory (LSTM) networks are a type of Recurrent Neural Network (RNN) that are particularly well suited for time series forecasting. LSTMs are able to capture long-term dependencies in time series data by using memory cells that can retain information for extended periods of time.

LSTMs take in the historical time series data as input, and use this data to learn the underlying patterns and relationships in the data. Once trained, the LSTM can then be used to make predictions about future events in the time series.

First, load the necessary libraries

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator

Then, load the data and prepare it for the LSTM model. This typically involves splitting the data into training and testing sets, scaling the data, and reshaping it into the format that the LSTM model expects. Full code are copied and pasted from this Github script https://github.com/nachi-hebbar/Time-Series-Forecasting-LSTM/blob/main/RNN_Youtube.ipynb

In [ ]:
train = train.set_index('date')
test = test.set_index('date')

In [ ]:
train.reset_index()

scaler = MinMaxScaler()
train.head(), test.head()

In [ ]:
scaler.fit(train)
scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)

In [ ]:
scaled_train[:10]

In [ ]:
# We do the same thing, but now instead for 12 months
n_input = 4
n_features = 1
generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=1)

# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

model.summary()

In [ ]:
# fit model
model.fit(generator, epochs=50)

In [ ]:
loss_per_epoch = model.history.history['loss']
plt.plot(range(len(loss_per_epoch)),loss_per_epoch)

In [ ]:
last_train_batch = scaled_train[-4:]
last_train_batch = last_train_batch.reshape((1, n_input, n_features))
model.predict(last_train_batch)

In [ ]:
scaled_test[0]

In [ ]:
test_predictions = []

first_eval_batch = scaled_train[-n_input:]
current_batch = first_eval_batch.reshape((1, n_input, n_features))

for i in range(len(test)):
    
    # get the prediction value for the first batch
    current_pred = model.predict(current_batch)[0]
    
    # append the prediction into the array
    test_predictions.append(current_pred) 
    
    # use the prediction to update the batch and remove the first value
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)

In [ ]:
test_predictions

In [ ]:
test.head()

In [ ]:
true_predictions = scaler.inverse_transform(test_predictions)
test['Predictions'] = true_predictions
test.plot(figsize=(20,8))

# 4. Exercise: Featured Competition

Let's challenge ourselves and apply these forecasting models we have learned on the featured competition dataset **GoDaddy - Microbusiness Density Forecasting.**

In [ ]:
# Load data
df_daddy = pd.read_csv('/kaggle/input/godaddy-microbusiness-density-forecasting/train.csv')

# Convert to datetime
df_daddy['first_day_of_month'] = pd.to_datetime(df_daddy['first_day_of_month'], format = "%Y-%m-%d")

# Rename columns
_ = df_daddy[['first_day_of_month', 'microbusiness_density']]
_.rename(columns = {'first_day_of_month':'date', 'microbusiness_density':'density',}, inplace = True)
_.head()

In [ ]:
# Group by month
df_grouped_daddy = _.groupby('date')['density'].mean().reset_index()
df_grouped_daddy.head()

In [ ]:
# Plot the graph
fig = plt.figure(figsize = (20, 8))
sns.lineplot(x = df_grouped_daddy['date'], y= df_grouped_daddy['density'], linewidth=2, )
plt.title("Average Monthly Microbusiness Density",
          fontsize = 18,
          fontweight = 'bold',
          fontfamily = 'serif',
          loc = 'center')

In [ ]:
# Run combinations of ARIMA(p,d,q)
model_fit = auto_arima(df_grouped_daddy['density'],
                       test='adf', # Adfuller test if the data is stationary or not
#                        m=36,
                       d=1,
                       D=1,
                       start_p=0,
                       start_q=0,
                       max_order=None,                       
                       max_p=7,
                       max_q=7,
                       max_d=2,
                       max_P=4,
                       max_Q=4,
                       max_D=2,
                       maxiter = 50,
                       alpha = 0.05,
#                        seasonal=True,
                       trace=True,
                       error_action='ignore',  
                       suppress_warnings=True, 
                       stepwise=True
                      )

model_fit.summary()

In [ ]:
model_ARIMA

In [ ]:
# Fit the ARIMA model
model_ARIMA = ARIMA(df_grouped_daddy['density'],
              order=(0,1,0),
              seasonal_order=(0, 0, 0, 0),
            )

# Fit the model
model_ARIMA = model_ARIMA.fit()

df_grouped_daddy['forecast_ARIMA'] = model_ARIMA.predict()
df_grouped_daddy[['density','forecast_ARIMA']].plot(figsize=(20,8))